In [137]:
# SQL functions
import pymysql
endpoint = 'ta10projectinstance1.c0m59tujlbqm.ap-southeast-2.rds.amazonaws.com'
username = 'admin'
password = 'fit5120ta10'
database_name = 'fit5120_i3_schema'
client_flag = 'CLIENT.MULTI_STATEMENTS'

connection = pymysql.connect(host = endpoint, user = username, passwd = password, db = database_name)

# Function used to clear all entries in a single table considering no indexes or Foreign keys are present
def truncate_handler(table):
    cursor = connection.cursor()
    cursor.execute("""TRUNCATE {}""".format(table))
    connection.commit()
    return 'table {} has been truncated'.format(table)

def country_region_handler(values):
    query = """INSERT INTO country_region (country_region_id, country_code, region_name) VALUES (%s, %s, %s)"""

    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def country_handler(values):
    query = """INSERT INTO country (country_name, country_code) VALUES (%s, %s)"""

    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def symptom_handler(values):
    query = """INSERT INTO symptom (symp_id, symp_name) VALUES (%s, %s)"""

    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def case_details_handler(values):
    query = """INSERT INTO case_details (case_id, symp_id) VALUES (%s, %s)"""

    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def case_handler(values):
    query = """INSERT INTO `case` (case_id, case_status, case_date, case_gender, country_region_id, case_death_date) VALUES (%s, %s, %s, %s, %s, %s)"""
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def prediction_handler(values):
    query = """INSERT INTO predictions (country_region_id, pred_date, pred_count) VALUES (%s, %s, %s)"""

    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'predictions table has been updated'

def query(query):
    sql_query = """{}""".format(query)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    return cursor.fetchall()

In [197]:
import pandas as pd
import datetime as dt
# Function used to load new datasource
def load_data():
    # Loads the data
    df = pd.read_csv('https://raw.githubusercontent.com/owid/monkeypox/main/owid-monkeypox-data.csv')
    # converts the date column to a datetime type
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d').dt.date
    # Drops the rows related to the global count
    df = df.loc[df['iso_code'] != 'OWID_WRL']
    return df

# Function used to check if a death has occured to populate case_death_date column
def death_date_check(row):
    if row['new_deaths'] == 0:
        return None
    else:
        return row['date']
# Function used to generate the country_region_id as the data is country level
def country_region_id_gen(row):
    c_r_id = row['iso_code'] + "_0"
    return c_r_id
# Function used to generate the case_id from the raw data
def case_id_gen(df):
    owid_entries = query("SELECT count(case_id) FROM `case` WHERE case_id LIKE 'OWID%'")[0][0]
    new_entries = df.shape[0]
    c_id = ['OWID_' + str(owid_entries + x) for x in range(0,new_entries)]
    return c_id

# Function used to extract the latest cases from the new dataset which have not been inserted into the rdbs yet
def new_cases(df, last_entry):
    # Selects all cases that have occured from a given date on wards
    df = df.loc[df['date'] > last_entry]
    # drops all columns which are not required to reduce computational loading
    df = df[['location', 'iso_code', 'date', 'new_deaths']]
    # Creates a new column for case_death_date
    df['case_death_date'] = df.apply(death_date_check, axis = 1)
    # Creates a new column for case_gender
    df['case_gender'] = None
    # Creates the country_region_id
    df['country_region_id'] = df.apply(country_region_id_gen, axis = 1)
    # Creates a case_id
    df['case_id'] = case_id_gen(df)
    # Populates the status column
    df['case_status'] = 'confirmed'
    # Renames all columns to the appropriate column names for importing
    df = df.rename({'date': 'case_date'}, axis = 1)
    # Selects a subset of the dataframe after all columns have been generated for the case table
    df = df[['case_id', 'case_status', 'case_date', 'case_gender', 'country_region_id', 'case_death_date']]
    return df

# Function used to find the new rows to be uploaded
def update_case(df):
    query('SELECT count(case_id) FROM `case` WHERE case_id LIKE "OWID%"')[0][0]

    return df
# Loads the new data
mpx_data = load_data()
# loads all data from a specified date onwards
#rdbs_last_rec = query("SELECT MAX(STR_TO_DATE(case_date, '%Y-%m-%d')) FROM `case`")[0][0]
mpx_latest_cases = new_cases(mpx_data, dt.date(2022, 9, 22))
mpx_update_case = update_case(mpx_latest_cases)

In [184]:
query('SELECT count(case_id) FROM `case` WHERE case_id LIKE "OWID%"')[0][0]

545

In [213]:
mpx_latest_cases.sort_values(['case_date', 'country_region_id'])

,case_id,case_status,case_date,case_gender,country_region_id,case_death_date
60,OWID_545,confirmed,2022-09-23,None,AND_0,None
180,OWID_553,confirmed,2022-09-23,None,ARG_0,None
338,OWID_561,confirmed,2022-09-23,None,AUS_0,None
466,OWID_566,confirmed,2022-09-23,None,AUT_0,None
658,OWID_574,confirmed,2022-09-23,None,BEL_0,None
...,...,...,...,...,...,...
7924,OWID_1043,confirmed,2022-09-30,None,SWE_0,None
8208,OWID_1059,confirmed,2022-09-30,None,TUR_0,None
8224,OWID_1067,confirmed,2022-09-30,None,UKR_0,None
8553,OWID_1083,confirmed,2022-09-30,None,USA_0,None


In [209]:
mpx_latest_cases.tail()

,case_id,case_status,case_date,case_gender,country_region_id,case_death_date
8609,OWID_1085,confirmed,2022-09-24,None,URY_0,None
8610,OWID_1086,confirmed,2022-09-25,None,URY_0,None
8611,OWID_1087,confirmed,2022-09-26,None,URY_0,None
8612,OWID_1088,confirmed,2022-09-27,None,URY_0,None
8613,OWID_1089,confirmed,2022-09-28,None,URY_0,None


In [198]:
mpx_update_case

,case_id,case_status,case_date,case_gender,country_region_id,case_death_date
60,OWID_545,confirmed,2022-09-23,None,AND_0,None
61,OWID_546,confirmed,2022-09-24,None,AND_0,None
62,OWID_547,confirmed,2022-09-25,None,AND_0,None
63,OWID_548,confirmed,2022-09-26,None,AND_0,None
64,OWID_549,confirmed,2022-09-27,None,AND_0,None
...,...,...,...,...,...,...
8609,OWID_1085,confirmed,2022-09-24,None,URY_0,None
8610,OWID_1086,confirmed,2022-09-25,None,URY_0,None
8611,OWID_1087,confirmed,2022-09-26,None,URY_0,None
8612,OWID_1088,confirmed,2022-09-27,None,URY_0,None


In [182]:
mpx_update_case

,location,iso_code,date,total_cases,total_deaths,new_cases,new_deaths,new_cases_smoothed,new_deaths_smoothed,new_cases_per_million,total_cases_per_million,new_cases_smoothed_per_million,new_deaths_per_million,total_deaths_per_million,new_deaths_smoothed_per_million
0,Andorra,AND,2022-07-25,2.0,0.0,2.0,0.0,0.29,0.0,25.306,25.306,3.669,0.0,0.0,0.0
1,Andorra,AND,2022-07-26,3.0,0.0,1.0,0.0,0.43,0.0,12.653,37.958,5.441,0.0,0.0,0.0
2,Andorra,AND,2022-07-27,3.0,0.0,0.0,0.0,0.43,0.0,0.000,37.958,5.441,0.0,0.0,0.0
3,Andorra,AND,2022-07-28,3.0,0.0,0.0,0.0,0.43,0.0,0.000,37.958,5.441,0.0,0.0,0.0
4,Andorra,AND,2022-07-29,3.0,0.0,0.0,0.0,0.43,0.0,0.000,37.958,5.441,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8706,Venezuela,VEN,2022-09-14,3.0,0.0,0.0,0.0,0.00,0.0,0.000,0.106,0.000,0.0,0.0,0.0
8707,Venezuela,VEN,2022-09-15,3.0,0.0,0.0,0.0,0.00,0.0,0.000,0.106,0.000,0.0,0.0,0.0
8708,Venezuela,VEN,2022-09-16,3.0,0.0,0.0,0.0,0.00,0.0,0.000,0.106,0.000,0.0,0.0,0.0
8709,Venezuela,VEN,2022-09-17,3.0,0.0,0.0,0.0,0.00,0.0,0.000,0.106,0.000,0.0,0.0,0.0


In [177]:
country_region_handler(('AUS_0','AUS',None))

"import successful ('AUS_0', 'AUS', None)"

In [143]:
mpx_latest_cases.sort_values('case_date')

,case_id,case_status,case_date,case_gender,country_region_id,case_death_date
60,OWID_0,confirmed,2022-09-23,None,AND_0,None
4128,OWID_263,confirmed,2022-09-23,None,ISL_0,None
4358,OWID_271,confirmed,2022-09-23,None,IRL_0,None
4491,OWID_279,confirmed,2022-09-23,None,ISR_0,None
4626,OWID_287,confirmed,2022-09-23,None,ITA_0,None
...,...,...,...,...,...,...
1024,OWID_67,confirmed,2022-09-30,None,BRA_0,None
5051,OWID_320,confirmed,2022-09-30,None,LTU_0,None
5158,OWID_328,confirmed,2022-09-30,None,LUX_0,None
4633,OWID_294,confirmed,2022-09-30,None,ITA_0,None


In [151]:
# Uploads table for case
case_df = mpx_latest_cases.copy()
for row in range(0,case_df.shape[0]):
    con1 = False
    con2 = False
    in_tuple = tuple(case_df.iloc[row, :])
    if type(in_tuple[3]) != str:
        con1 = True
    if type(in_tuple[5]) != str:
        con2 = True
    if con1 == True and con2 == True:
        case_handler((in_tuple[0], in_tuple[1], in_tuple[2], None, in_tuple[4], None))
    elif con1 == True and con2 == False:
        case_handler((in_tuple[0], in_tuple[1], in_tuple[2], None, in_tuple[4], in_tuple[5]))
    elif con1 == False and con2 == True:
        case_handler((in_tuple[0], in_tuple[1], in_tuple[2], in_tuple[3], in_tuple[4], None))
    else: 
        case_handler(in_tuple)

In [176]:
query("SELECT count(case_id) FROM `case`")[0][0]

65760

In [150]:
query("SELECT COUNT(case_id) FROM `case` WHERE country_region_id LIKE 'AUS%'")[0][0]

132

In [153]:
query("SELECT COUNT(case_id) FROM `case` WHERE country_region_id LIKE 'AUS%'")[0][0]

137

In [156]:
query("SELECT MAX(STR_TO_DATE(case_date, '%Y-%m-%d')) FROM `case`")[0][0]

datetime.date(2022, 9, 30)

In [162]:
query("SELECT TO_CHAR (MAX(STR_TO_DATE(case_date, '%Y-%m-%d'))) FROM `case` WHERE country_region_id LIKE 'AUS%'")[0][0]

'2022-09-27 00:00:00'

In [178]:
query("""
(SELECT country_region_id FROM country_region
WHERE country_code = (SELECT country_code FROM country WHERE country_name = 'Australia'))
""")

(('AUS_0',),
 ('AUS_1',),
 ('AUS_2',),
 ('AUS_3',),
 ('AUS_4',),
 ('AUS_5',),
 ('AUS_6',))

In [180]:
query("""
SELECT count(case_id)-1 AS 'cases in australia' FROM fit5120_i3_schema.case AS c1 RIGHT JOIN
(SELECT country_region_id FROM country_region
WHERE country_code = (SELECT country_code FROM country WHERE country_name = 'Australia')) AS c2
ON c1.country_region_id = c2.country_region_id;
""")

((136,),)

In [ ]:
query("""
(SELECT country_region_id FROM country_region
WHERE country_code = (SELECT country_code FROM country WHERE country_name = 'Australia'))
""")

In [57]:
rdbs_last_rec = query("SELECT MAX(STR_TO_DATE(case_date, '%Y-%m-%d')) FROM `case`")[0][0]
mpx_data.loc[mpx_data['date'] > rdbs_last_rec]

,location,iso_code,date,total_cases,total_deaths,new_cases,new_deaths,new_cases_smoothed,new_deaths_smoothed,new_cases_per_million,total_cases_per_million,new_cases_smoothed_per_million,new_deaths_per_million,total_deaths_per_million,new_deaths_smoothed_per_million
60,Andorra,AND,2022-09-23,4.0,0.0,0.0,0.0,0.00,0.00,0.000,50.611,0.000,0.0,0.000,0.0
61,Andorra,AND,2022-09-24,4.0,0.0,0.0,0.0,0.00,0.00,0.000,50.611,0.000,0.0,0.000,0.0
62,Andorra,AND,2022-09-25,4.0,0.0,0.0,0.0,0.00,0.00,0.000,50.611,0.000,0.0,0.000,0.0
63,Andorra,AND,2022-09-26,4.0,0.0,0.0,0.0,0.00,0.00,0.000,50.611,0.000,0.0,0.000,0.0
64,Andorra,AND,2022-09-27,4.0,0.0,0.0,0.0,0.00,0.00,0.000,50.611,0.000,0.0,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8859,World,OWID_WRL,2022-09-26,65600.0,24.0,128.0,0.0,520.71,0.43,0.016,8.294,0.066,0.0,0.003,0.0
8860,World,OWID_WRL,2022-09-27,66472.0,24.0,872.0,0.0,565.71,0.14,0.110,8.404,0.072,0.0,0.003,0.0
8861,World,OWID_WRL,2022-09-28,67556.0,25.0,1084.0,1.0,477.29,0.14,0.137,8.541,0.060,0.0,0.003,0.0
8862,World,OWID_WRL,2022-09-29,67939.0,25.0,383.0,0.0,484.43,0.14,0.048,8.590,0.061,0.0,0.003,0.0


In [58]:
mpx_data.iso_code.unique()

array(['AND', 'ARG', 'ABW', 'AUS', 'AUT', 'BHS', 'BHR', 'BRB', 'BEL',
       'BEN', 'BMU', 'BOL', 'BIH', 'BRA', 'BGR', 'CMR', 'CAN', 'CAF',
       'CHL', 'CHN', 'COL', 'COG', 'CRI', 'HRV', 'CUB', 'CUW', 'CYP',
       'CZE', 'COD', 'DNK', 'DOM', 'ECU', 'EGY', 'SLV', 'EST', 'FIN',
       'FRA', 'GEO', 'DEU', 'GHA', 'GIB', 'GRC', 'GRL', 'GLP', 'GUM',
       'GTM', 'GUY', 'HND', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRL',
       'ISR', 'ITA', 'JAM', 'JPN', 'JOR', 'LVA', 'LBN', 'LBR', 'LTU',
       'LUX', 'MLT', 'MTQ', 'MEX', 'MDA', 'MCO', 'MNE', 'MAR', 'NLD',
       'NCL', 'NZL', 'NGA', 'NOR', 'PAN', 'PRY', 'PER', 'PHL', 'POL',
       'PRT', 'PRI', 'QAT', 'ROU', 'RUS', 'MAF', 'SAU', 'SRB', 'SGP',
       'SVK', 'SVN', 'ZAF', 'KOR', 'ESP', 'SDN', 'SWE', 'CHE', 'THA',
       'TUR', 'UKR', 'ARE', 'GBR', 'USA', 'URY', 'VEN', 'OWID_WRL'],
      dtype=object)

In [49]:
query("SELECT DISTINCT case_date FROM `case` WHERE STR_TO_DATE(case_date, '%Y-%m-%d') >= STR_TO_DATE('2022-09-22', '%Y-%m-%d')")[0][0]

'2022-09-22'

In [46]:
ts = dt.datetime('2022-9-23', format='%Y-%m-%d')

TypeError: 'str' object cannot be interpreted as an integer

In [45]:
mpx_data.loc[]

,location,iso_code,date,total_cases,total_deaths,new_cases,new_deaths,new_cases_smoothed,new_deaths_smoothed,new_cases_per_million,total_cases_per_million,new_cases_smoothed_per_million,new_deaths_per_million,total_deaths_per_million,new_deaths_smoothed_per_million
0,Andorra,AND,2022-07-25,2.0,0.0,2.0,0.0,0.29,0.00,25.306,25.306,3.669,0.0,0.000,0.0
1,Andorra,AND,2022-07-26,3.0,0.0,1.0,0.0,0.43,0.00,12.653,37.958,5.441,0.0,0.000,0.0
2,Andorra,AND,2022-07-27,3.0,0.0,0.0,0.0,0.43,0.00,0.000,37.958,5.441,0.0,0.000,0.0
3,Andorra,AND,2022-07-28,3.0,0.0,0.0,0.0,0.43,0.00,0.000,37.958,5.441,0.0,0.000,0.0
4,Andorra,AND,2022-07-29,3.0,0.0,0.0,0.0,0.43,0.00,0.000,37.958,5.441,0.0,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8859,World,OWID_WRL,2022-09-26,65600.0,24.0,128.0,0.0,520.71,0.43,0.016,8.294,0.066,0.0,0.003,0.0
8860,World,OWID_WRL,2022-09-27,66472.0,24.0,872.0,0.0,565.71,0.14,0.110,8.404,0.072,0.0,0.003,0.0
8861,World,OWID_WRL,2022-09-28,67556.0,25.0,1084.0,1.0,477.29,0.14,0.137,8.541,0.060,0.0,0.003,0.0
8862,World,OWID_WRL,2022-09-29,67939.0,25.0,383.0,0.0,484.43,0.14,0.048,8.590,0.061,0.0,0.003,0.0
